Install ipython notebook widgets:

https://ipywidgets.readthedocs.io/en/latest/user_install.html

In [7]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle code"></form>''')

In [2]:
import sys
sys.path.append('..')

In [15]:
import gdoc_revisions
from oauth2client.service_account import ServiceAccountCredentials
import ipywidgets as widgets
from IPython.display import display, clear_output

CREDENTIAL_FILE = '../settings/doc-revisions-98c56004e106.json'
scope = ['https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE, scope)
# FILE_ID = '1aSAA-ZA8bGvJSpgFhVgJu89EDMPMRm5IrcdeV85JmJE'
# FILE_ID = '1fiFUuGuUeU6W-JrbNFSfZIwc6U8I0vyyypnS5zNHiZg'
FILE_ID = '1OApuxAtRIq4MLqMl175jiL_mIKeCMXA6AqFfgT7m68M'

# doc-revisions@appspot.gserviceaccount.com

In [18]:
gdoc = gdoc_revisions.GoogleDoc(FILE_ID, credentials)

slider = widgets.IntSlider(min=1, max=len(gdoc.revisions))

def update_document_snapshot(revision_id_change):
    clear_output(wait=True)
    revision_id = revision_id_change['new']
    print gdoc.at_revision(revision_id).render()

slider.observe(update_document_snapshot, names='value')

In [17]:
import pandas as pd

# create pandas dataframe of revisions
df_revisions = pd.DataFrame([r.to_dict() for r in gdoc.revisions])

operations = []
for revision in gdoc.revisions:
    for operation in revision.operations:
        operations.append({
            'revision_id':revision.revision_id,
            'operation': operation
        })
df_operations = (
    df_revisions[['revision_id','time','user_id']]
    .merge(pd.DataFrame(operations))
)

df_revisions.head()[['revision_id','time','user_id','operation_raw',]]

,revision_id,time,user_id,operation_raw
0,1,2016-12-18 21:08:21.530,15537773617043190792,"{u'mts': [{u'ei': 1, u'ty': u'as', u'st': u'pa..."
1,2,2016-12-18 21:09:27.936,15537773617043190792,{u'ty': u'null'}
2,3,2016-12-18 21:09:55.602,15537773617043190792,"{u'ibi': 1, u's': u'Vp', u'ty': u'is'}"
3,4,2016-12-18 21:09:56.300,15537773617043190792,"{u'ibi': 3, u's': u'a', u'ty': u'is'}"
4,5,2016-12-18 21:09:58.100,15537773617043190792,"{u'ibi': 4, u's': u'l', u'ty': u'is'}"


In [19]:
display(slider)